<div style="font-size: 14pt;">Prof. Krzysztof Rybinski</div><br/><br/>
<div style="font-size: 22pt;"><b>Artificial Intelligence course</b></div><br/><br/>
<div style="font-size: 18pt;">LAB 11.1</div><br/>
<div style="font-size: 18pt;">- Predicting course ratings, RMP dataset</div><br/><br/>

In [ ]:
import torch

In [ ]:
#exetute this cell only in Google Colab
!pip install transformers
!pip install datasets

In [ ]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import Trainer
from datasets import load_metric
from sklearn.model_selection import train_test_split

In [ ]:
#exetute this cell only in Google Colab
from google.colab import files
import io

In [ ]:
class RMPDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
#exetute this cell only in Google Colab
uploaded = files.upload()
rmp = pd.read_csv(io.BytesIO(uploaded['rmp_small.csv']))

In [ ]:
rmp = pd.read_csv("data/rmp_small.csv") #for local drive

In [ ]:
rmp

In [ ]:
sentences = list(rmp['review'])
labels = list(rmp['score'])

In [ ]:
#remove nan from both lists
selnan = list()
for idx,name in enumerate(sentences):
    if name == '' or pd.isnull(name):
        selnan.append(idx)
sentences = [sentences[i] for i, e in enumerate(sentences) if i not in selnan]
labels = [labels[i] for i, e in enumerate(labels) if i not in selnan]
(len(sentences), len(labels))

In [ ]:
#skip for now, will do it later
#reduce the size of the dataset to 10%
import random
no_items = len(sentences)
no_items_sample = int(np.floor(0.1*no_items))
seq_rand = random.sample(list(range(no_items)), no_items_sample)
len(seq_rand)

In [ ]:
#skip for now, will do it later
sentences = [sentences[i] for i in seq_rand]
labels = [labels[i] for i in seq_rand]
len(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sentences, labels, test_size=0.1, random_state=42, shuffle=True)

In [ ]:
checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
train_encodings = tokenizer(X_train, truncation=True, padding=True)
#val_encodings = tokenizer(X_val, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
train_dataset = RMPDataset(train_encodings, y_train)
#val_dataset = RMPDataset(val_encodings, y_val)
test_dataset = RMPDataset(test_encodings, y_test)

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
training_args = TrainingArguments("RMP_train_args",
    per_device_train_batch_size=32,# batch size per device during training
#    per_device_eval_batch_size=64,   # batch size for evaluation
    evaluation_strategy="no",
    save_strategy="no",
    num_train_epochs=1
 )

training_args

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=5)

trainer = Trainer(
    model,
    training_args,
    train_dataset=train_dataset,
#    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(test_dataset)
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
predictions.predictions[:10]

In [ ]:
predictions.label_ids[:10]

In [ ]:
compute_metrics([predictions.predictions, predictions.label_ids])

How do I download the trained model from Google Colab?

    Compress the directory where you stored your model - Make sure it contains nothing else: ! tar -czvf model.tar.gz models/
    Download the model: from google.colab import files files.download('model.tar.gz')
        
Or check this post
https://medium.com/@ml_kid/how-to-save-our-model-to-google-drive-and-reuse-it-2c1028058cb2